In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import os
os.listdir('sample_data')
import pandas as pd

In [18]:
Tweets = pd.read_csv('sample_data/Tweets.csv')
Tweets.tail() #melihat detail 5 data terakhir

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
14635,569587686496825344,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different f...,NaN,2015-02-22 12:01:01 -0800,NaN,NaN
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flig...,NaN,2015-02-22 11:59:46 -0800,Texas,NaN
14637,569587242672398336,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to...,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN
14638,569587188687634433,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my ...",NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada)
14639,569587140490866689,neutral,0.6771,NaN,0.0000,American,NaN,daviddtwu,NaN,0,@AmericanAir we have 8 ppl so we need 2 know h...,NaN,2015-02-22 11:58:51 -0800,"dallas, TX",NaN


In [19]:
# menjalankan proses one-hot-encoding dan membuat df baru
del Tweets['tweet_id']
del Tweets['retweet_count']
del Tweets['tweet_created']
del Tweets['tweet_location']
Tweets.rename(columns={'airline_sentiment': 'sentiment'}, inplace=True) #mengubah nama kolom df
category = pd.get_dummies(Tweets.sentiment)
df_new = pd.concat([Tweets, category], axis=1)
df_new = df_new.drop(columns='sentiment')
df_new

,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,text,tweet_coord,user_timezone,negative,neutral,positive
0,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,@VirginAmerica What @dhepburn said.,NaN,Eastern Time (US & Canada),0,1,0
1,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,@VirginAmerica plus you've added commercials t...,NaN,Pacific Time (US & Canada),0,0,1
2,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,@VirginAmerica I didn't today... Must mean I n...,NaN,Central Time (US & Canada),0,1,0
3,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,@VirginAmerica it's really aggressive to blast...,NaN,Pacific Time (US & Canada),1,0,0
4,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,@VirginAmerica and it's a really big bad thing...,NaN,Pacific Time (US & Canada),1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,@AmericanAir thank you we got on a different f...,NaN,NaN,0,0,1
14636,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,@AmericanAir leaving over 20 minutes Late Flig...,NaN,NaN,1,0,0
14637,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,@AmericanAir Please bring American Airlines to...,NaN,NaN,0,1,0
14638,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,"@AmericanAir you have my money, you change my ...",NaN,Eastern Time (US & Canada),1,0,0


In [22]:
# mengubah nilai dari df kedalam tipe data np
text = df_new['text'].values
label = df_new[['negative', 'neutral', 'positive']].values

# membagi dataset
text_training, text_test, label_training, label_test = train_test_split(text, label, test_size=0.2)

In [23]:
#tokenisasi agar teks dipahami model
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(text_training)
tokenizer.fit_on_texts(text_test)

sequence_training = tokenizer.texts_to_sequences(text_training)
sequence_test = tokenizer.texts_to_sequences(text_test)

padded_training = pad_sequences(sequence_training)
padded_test = pad_sequences(sequence_test)

In [24]:
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
                             tf.keras.layers.LSTM(64),
                             tf.keras.layers.Dense(128, activation='relu'),
                             tf.keras.layers.Dense(64, activation='relu'),
                             tf.keras.layers.Dense(3, activation='softmax')
])

In [25]:
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [26]:
# melatih model
num_epochs = 40
history = model.fit(padded_training, label_training, epochs=num_epochs,
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/40
366/366 - 10s - loss: 0.6996 - accuracy: 0.6963 - val_loss: 0.5878 - val_accuracy: 0.7579
Epoch 2/40
366/366 - 7s - loss: 0.4814 - accuracy: 0.8061 - val_loss: 0.5250 - val_accuracy: 0.7995
Epoch 3/40
366/366 - 7s - loss: 0.3688 - accuracy: 0.8582 - val_loss: 0.5543 - val_accuracy: 0.7988
Epoch 4/40
366/366 - 7s - loss: 0.2948 - accuracy: 0.8910 - val_loss: 0.5603 - val_accuracy: 0.7869
Epoch 5/40
366/366 - 7s - loss: 0.2472 - accuracy: 0.9115 - val_loss: 0.6519 - val_accuracy: 0.7971
Epoch 6/40
366/366 - 7s - loss: 0.2004 - accuracy: 0.9308 - val_loss: 0.6906 - val_accuracy: 0.7794
Epoch 7/40
366/366 - 7s - loss: 0.1671 - accuracy: 0.9425 - val_loss: 0.7328 - val_accuracy: 0.7715
Epoch 8/40
366/366 - 7s - loss: 0.1396 - accuracy: 0.9520 - val_loss: 0.8439 - val_accuracy: 0.7790
Epoch 9/40
366/366 - 7s - loss: 0.1209 - accuracy: 0.9600 - val_loss: 0.9601 - val_accuracy: 0.7780
Epoch 10/40
366/366 - 7s - loss: 0.0989 - accuracy: 0.9671 - val_loss: 1.0655 - val_accuracy: 0.775